In [6]:
#   22i-0792 Assignment 4

In [46]:
#   Solving Kirchoffs System, For Desnse I used Pivoted Gaussian Elimination and for sparce i used Scipy's Sparce

In [39]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg
import time


In [40]:

# Helper function to swap rows
def rowSwap(v,i,j):
    if len(v.shape) == 1:
        v[i],v[j] = v[j],v[i]
    else:
        v[[i,j],:] = v[[j,i],:]

# Gaussian Elimination with Pivoting
def pGauss_elimination(A, b, tol=1.0e-15):
    n = len(b)
    for k in range(n-1):
        # Swap rows if necessary (partial pivoting)
        p = np.argmax(np.abs(A[k:n,k])) + k
        if np.abs(A[p,k]) < tol: 
            raise ValueError("Singular matrix has been detected")
        if p != k:
            rowSwap(b,k,p)
            rowSwap(A,k,p)
        # Perform Gauss elimination
        for i in range(k+1,n):
            if A[k,k] != 0.0:
                mik = A[i,k]/A[k,k]
                A[i,k+1:n] = A[i,k+1:n] - mik*A[k,k+1:n]
                b[i] = b[i] - mik*b[k]
    # Backward substitution
    if np.abs(A[n-1,n-1]) < tol:
        raise ValueError("Singular matrix has been detected")        
    b[n-1] = b[n-1]/A[n-1,n-1]
    for k in range(n-2,-1,-1):
        b[k] = (b[k] - np.dot(A[k,k+1:n], b[k+1:n])) / A[k,k]
    return b


In [41]:

# Coefficient matrix A
A = np.array([
    [1, -1, -1,  0,  0,  0,  0,  0],  # I0 = I1 + I2
    [0,  1,  0, -1,  0, -1,  0,  0],  # I1 = I3 + I5
    [0,  0,  1,  0, -1,  0, -1,  0],  # I2 = I4 + I6
    [0,  0,  0,  0,  0,  1,  1, -1],  # I7 = I5 + I6
    [2,  1,  0,  0,  0,  0,  0,  0],  # 20 = 2I0 + I1
    [0, -1,  0,  1,  2,  0,  0,  0],  # 0 = I3 + 2I4 - I1
    [0,  0,  0, -2,  0,  1,  0, -1],  # 0 = I5 - 2I3 - I7
    [0,  0,  0,  0,  1,  0,  2, -1],  # 0 = I4 + 2I6 - I7
])

# Right-hand side vector B
B = np.array([0, 0, 0, 0, 20, 0, 0, 0])


In [42]:

# Part (b) Dense Matrix Solution with Gaussian Elimination
start_dense = time.time()
dense_solution = pGauss_elimination(A.copy(), B.copy())  # Use copy to avoid modifying original A and B
end_dense = time.time()

print("Dense Matrix Solution using Gaussian Elimination with Pivoting:")
print(dense_solution)
print(f"Dense Matrix Computation Time: {end_dense - start_dense:.6f} seconds")


Dense Matrix Solution using Gaussian Elimination with Pivoting:
[ 8  3  7 -2  2  5  5 10]
Dense Matrix Computation Time: 0.003726 seconds


In [43]:

# Part (c) Sparse Matrix Solution using SciPy
A_sparse = sparse.csc_matrix(A)  # Convert to sparse matrix (Compressed Sparse Column format)

start_sparse = time.time()
sparse_solution = linalg.spsolve(A_sparse, B)
end_sparse = time.time()

print("\nSparse Matrix Solution using SciPy:")
print(sparse_solution)
print(f"Sparse Matrix Computation Time: {end_sparse - start_sparse:.6f} seconds")



Sparse Matrix Solution using SciPy:
[ 8.23529412  3.52941176  4.70588235 -1.17647059  2.35294118  4.70588235
  2.35294118  7.05882353]
Sparse Matrix Computation Time: 0.001998 seconds


In [44]:

# Part (d) Compare Speeds
dense_time = end_dense - start_dense
sparse_time = end_sparse - start_sparse

print("\nComparison of Speed:")
print(f"Dense Method Time: {dense_time:.6f} seconds")
print(f"Sparse Method Time: {sparse_time:.6f} seconds")
print(f"Faster Method: {'Dense' if dense_time < sparse_time else 'Sparse'}")



Comparison of Speed:
Dense Method Time: 0.003726 seconds
Sparse Method Time: 0.001998 seconds
Faster Method: Sparse


In [45]:

# Part (e) Compute Difference Between Solutions
difference = np.abs(np.array(dense_solution) - sparse_solution)

print("\nDifference Between Solutions:")
print(difference)

# Check Accuracy
tolerance = 1e-6
if np.all(difference < tolerance):
    print(f"Both solutions are accurate within the tolerance of {tolerance}.")
else:
    print("Solutions differ beyond the acceptable tolerance.")



Difference Between Solutions:
[0.23529412 0.52941176 2.29411765 0.82352941 0.35294118 0.29411765
 2.64705882 2.94117647]
Solutions differ beyond the acceptable tolerance.
